In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import boto3
from dotenv import load_dotenv
import os
from xgboost import XGBClassifier
import pandas as pd
from mlflow.models.signature import infer_signature
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
load_dotenv(dotenv_path="../.env")

access_key = os.getenv("AWS_ACCESS_KEY_ID_CLIENT")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY_CLIENT")
s3_url = os.getenv("MLFLOW_S3_ENDPOINT_URL_CLIENT")
tracker_url = os.getenv("MLFLOW_URL_CLIENT")
os.environ["MLFLOW_S3_ENDPOINT_URL"] = os.getenv("MLFLOW_S3_ENDPOINT_URL_CLIENT")
os.environ["MLFLOW_TRACKING_USERNAME"] = os.getenv("MLFLOW_ADMIN_USERNAME_CLIENT")
os.environ["MLFLOW_TRACKING_PASSWORD"] = os.getenv("MLFLOW_ADMIN_PASSWORD_CLIENT")
os.environ["MLFLOW_S3_IGNORE_TLS"] = "true"
os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"
bucket_name = "dataset"
object_name = "water_potability.csv"

In [ ]:
s3 = boto3.client(
    's3',
    endpoint_url=s3_url,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    verify=False
)

In [ ]:
try:
    # Attempt to list buckets
    response = s3.list_buckets()
    
    # If successful, print bucket names
    print("Connected successfully! Buckets available:")
    for bucket in response['Buckets']:
        print(f"- {bucket['Name']}")
except Exception as e:
    # Print any connection errors
    print("Connection error:", e)


In [ ]:
try:
    response = s3.get_object(Bucket=bucket_name, Key=object_name)
    print(response)
    dataset_content = response.get('Body')
    print(dataset_content)
    df = pd.read_csv(dataset_content)
    print("Dataset loaded successfully:")
    print(df.head())
except Exception as e:
    print("Error fetching dataset from MinIO:", e)


In [ ]:
feature = ['ph','Solids','Turbidity','Potability']
# feature = ['ph','Solids','Potability']
# feature = ['ph','Potability']
df = df[feature]
df.dropna(inplace=True)

In [ ]:
df.describe()

In [ ]:
df[df['Potability']==1].describe()

In [ ]:
df[df['Potability']==0].describe()

In [ ]:
df[df['Potability']==0].sample(5)

In [ ]:
df[df['Potability']==1].sample(5)

In [ ]:
df['Potability'].value_counts()

In [ ]:
df.isna().sum()

In [ ]:
data = df.drop('Potability', axis=1)
target = df['Potability']

In [ ]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
data, target = smt.fit_resample(data, target)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
mlflow.set_tracking_uri(tracker_url)
experiment_name = "water_potability"
mlflow.set_experiment(experiment_name)

In [ ]:
tags = {
    "ph": "True",
    "Hardness": "False",
    "Solids": "True",
    "Chloramines": "False",
    "Sulfate": "False",
    "Conductivity": "False",
    "Organic_carbon": "False",
    "Trihalomethanes": "False",
    "Turbidity": "True",
    "Potability": "Binary",
}


In [ ]:
# Model parameters
n_estimators = 100
max_depth = 5
random_state = 42


with mlflow.start_run():
    # Log parameters
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)

    # Initialize and train the model
    model = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    model.fit(X_train, y_train)

    # Evaluate the model
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    mlflow.log_metric("accuracy", accuracy)

    # Define input example and signature
    input_example = X_test.iloc[:1].fillna(X_test.mean())
    signature = infer_signature(X_test, model.predict(X_test))

    # Log the model with MLflow
    mlflow.sklearn.log_model(
        model,
        artifact_path="model",
        input_example=input_example,
        signature=signature
    )

    # Print run information
    run_id = mlflow.active_run().info.run_id
    mlflow.set_tags(tags)
    mlflow.set_tag("Model", "XGBoost")
    print(f"Run ID: {run_id}")
    print(f"Model accuracy: {accuracy}")


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# Model parameters
max_iter = 1000
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter = max_iter))
])


with mlflow.start_run():
    # Log parameters
    mlflow.log_param("max_iter", max_iter)

    # Initialize and train the model
    pipeline.fit(X_train, y_train)

    # Evaluate the model
    predictions = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    mlflow.log_metric("accuracy", accuracy)

    # Define input example and signature
    input_example = X_test.iloc[:1].fillna(X_test.mean())
    signature = infer_signature(X_test, pipeline.predict(X_test))

    # Log the model with MLflow
    mlflow.sklearn.log_model(
        pipeline,
        artifact_path="model",
        input_example=input_example,
        signature=signature
    )

    # Print run information
    run_id = mlflow.active_run().info.run_id
    mlflow.set_tags(tags)
    mlflow.set_tag("Model", "Logistic Regression")
    print(f"Run ID: {run_id}")
    print(f"Model accuracy: {accuracy}")

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# Model parameters
n_estimators = 100
max_depth = 5
random_state = 42

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("rf", RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state))
])

with mlflow.start_run():
    # Log parameters
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)
    # Initialize and train the model
    pipeline.fit(X_train, y_train)

    # Evaluate the model
    predictions = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    mlflow.log_metric("accuracy", accuracy)

    # Define input example and signature
    input_example = X_test.iloc[:1].fillna(X_test.mean())
    signature = infer_signature(X_test, pipeline.predict(X_test))

    # Log the model with MLflow
    mlflow.sklearn.log_model(
        pipeline,
        artifact_path="model",
        input_example=input_example,
        signature=signature
    )

    # Print run information
    run_id = mlflow.active_run().info.run_id
    mlflow.set_tags(tags)
    mlflow.set_tag("Model", "Random Forest")
    print(f"Run ID: {run_id}")
    print(f"Model accuracy: {accuracy}")

In [ ]:
import mlflow.pyfunc

model_uri = f"runs:/{run_id}/model"
model = mlflow.pyfunc.load_model(model_uri)

# Check if metadata and signature are available
if model.metadata:
    print("Model signature:", model.metadata.get_input_schema())
else:
    print("Model metadata is missing.")